### Installing libraries

In [1]:
%%capture
!pip install langchain
!pip install transformers
!pip install unstructured
!pip install pypdf
!pip install llama-index==0.9.13
!pip install ipywidgets
!pip -qqq install bitsandbytes accelerate
!pip install pydantic==1.10.11
!pip install sentence-transformers
!pip install chromadb

### Importing libraries

In [3]:
from IPython.display import Markdown, display
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex
from llama_index.node_parser import SimpleNodeParser
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
import tiktoken
from llama_index.text_splitter import SentenceSplitter
import re
import chromadb
from llama_index import get_response_synthesizer
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor import LLMRerank
from llama_index.postprocessor import SentenceEmbeddingOptimizer
from llama_index.postprocessor import SentenceTransformerRerank
import torch
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate
import torch
from huggingface_hub import login
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
# load documents
documents = SimpleDirectoryReader("/content/data/").load_data()

In [5]:
len(documents)

144

In [6]:
documents[0].get_content()

'11/16/23, 9:24 PM AirMedia issues\nhttps://utoronto.sharepoint.com/sites/lsm-TalkmasterKB/SitePages/Troubleshooting-Guides/AirMedia-Issues.aspx 1/5TROUBLESHOOTING GUIDE\nAirMedia issues\nKB Home  > User T roubleshooting \xa0> AirMedia issues\nQuestions\nWhich model o f AirMedia r eceiv er does the room have : is it an AM-200  or an AM-3200 ?\nDo not\xa0follow the software download link on the AM-200 splash page\nWhat kind of device are they using: Mac, Windows ,\xa0iPad/iPhone  or Andr oid tablet/phone?\nHave they added the AirMedia receiver in this room as a "favourite" ,\xa0to make it easier to connect to in the\nfuture?\nActions\nHow\xa0t o install\xa0the AirMedia app on the user\' s device , then get\xa0their image up on the big screen\nMac (using the Chrome browser)\nWindows  instructions\niPad/iPhone  and Andr oid phone/tablet\nWhat to do if they\'re getting the error message: "Unable t o verify AirMedia cer tificat e"\nInstructions for finding & installing the newest version of

In [7]:
import unicodedata
def clean_text2(raw_text):
    #Make the text lowercase
    #Escape Unicode characters
    text = unicodedata.normalize('NFKD', raw_text).encode('ascii', 'ignore').decode('utf-8')
    text = raw_text.lower()

    #Remove pattern matching "troubleshooting guide" It is in the header
    text = re.sub(r'troubleshooting\s*guide', '', text, flags=re.IGNORECASE)

    #Remove dates in the format --/--/--
    text = re.sub(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b', '', text)

    #Remove time
    text = re.sub(r'\b\d{1,2}:\d{1,2}\s*(?:AM|PM|am|pm)?\b', '', text)

    #Remove URLs or links
    text = re.sub(r'https?://\S+|www\.\S+|<.*?>', '', text)

    #Remove page numbers like 1/2, 2/3, etc.
    text = re.sub(r'\d+/\d+', '', text)

    #Remove file naming convention
    text = re.sub(r'kb home', '', text)
    text = re.sub(r'user troubleshooting', '', text)
    text = re.sub(r'>', '', text)

    #Remove everything after "page owner" pattern
    page_owner_index = text.find('page owner')
    if page_owner_index != -1:
        text = text[:page_owner_index]


    #Remove everything after "page owner" pattern
    page_owner_index = text.find('changelog')
    if page_owner_index != -1:
        text = text[:page_owner_index]

    #Remove unwanted spaces
    text = ' '.join(text.split())

    return text

In [8]:
# clean documents
for doc in documents:
    doc.set_content(clean_text2(doc.get_content()))

In [9]:
len(documents)

144

In [10]:
documents[77].get_content()

', no image from user device on big screen'

In [11]:
text_splitter = SentenceSplitter(
  separator=" ", chunk_size=200, chunk_overlap=20,
  paragraph_separator="\n\n", secondary_chunking_regex="[^,.;。]+[,.;。]?",
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
doc_nodes = text_splitter.get_nodes_from_documents(documents)

In [14]:
len(doc_nodes)

282

In [15]:
doc_nodes[0].get_content()

', airmedia issues airmedia issues user t roubleshooting airmedia issues questions which model o f airmedia r eceiv er does the room have : is it an am-200 or an am-3200 ? do not follow the software download link on the am-200 splash page what kind of device are they using: mac, windows , ipad/iphone or andr oid tablet/phone? have they added the airmedia receiver in this room as a "favourite" , to make it easier to connect to in the future? actions how t o install the airmedia app on the user\' s device ,'

In [16]:
doc_nodes[1].get_content()

'actions how t o install the airmedia app on the user\' s device , then get their image up on the big screen mac (using the chrome browser) windows instructions ipad/iphone and andr oid phone/tablet what to do if they\'re getting the error message: "unable t o verify airmedia cer tificat e" instructions for finding & installing the newest version of the airmedia app (on a mac) instructions for finding & installing the newest version of the airmedia app (on windows) what to do if they entered the correct url for airmedia,'

In [18]:
# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [19]:
torch.cuda.empty_cache()

In [20]:
login("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx") ###Your huggingface apikey goes here

# Model names (make sure you have access on HF)
LLAMA2_7B_CHAT = "meta-llama/Llama-2-7b-chat-hf"

selected_model = LLAMA2_7B_CHAT

SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents.
The provided documents are knowledge base of solving some commonly occuring techincal issues in classrrooms. You will help humans to solve those AV issues.
Tell max to max 3 possible solutions for the given question.
 Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help,etc. Just answer directly.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 1, "do_sample": True, "top_p":1},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto",
    # change these settings below depending on your GPU
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True}
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [21]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("talkmaster-knowledge-base")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm = llm)
index = VectorStoreIndex(doc_nodes, storage_context=storage_context, service_context=service_context)

# Saving the index is done. Now go to loading the index and later parts


### Uncomment and run if you have a index saved on your disk

In [22]:
# # load from disk
# db2 = chromadb.PersistentClient(path="./chroma_db")
# service_context = ServiceContext.from_defaults(embed_model=embed_model, llm = llm)
# chroma_collection = db2.get_or_create_collection("talkmaster-knowledge-base") ### Write your collection name
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# index = VectorStoreIndex.from_vector_store(
#     vector_store,
#     service_context=service_context,
# )

In [23]:
# delete the last document
print("count before", chroma_collection.count())

count before 282


In [24]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5
)

In [25]:
# Configure response synthesizer
response_synthesizer = get_response_synthesizer(service_context=service_context, response_mode='refine')

In [29]:
# We choose a model with relatively high speed and decent accuracy.
postprocessors = [SentenceEmbeddingOptimizer(percentile_cutoff=0.5,embed_model = embed_model), SentenceTransformerRerank(model="cross-encoder/ms-marco-MiniLM-L-12-v2", top_n=3)]
query_engine = index.as_query_engine(response_synthesizer=response_synthesizer, node_postprocessors=postprocessors)

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [30]:
# Execute the query
response = query_engine.query("I am in MP137 and I am using a Mac laptop. I am connected to hdmi 1 \
                               and the proejector is not projecting what's on my screen. What should I do?")
display(Markdown(f"<b>{response}</b>"))

<b>Given the additional information provided, here are three possible solutions for the issue of no image from the user device on the big screen: 

1. Remotely press the display on/power button and listen for beeps: Try pressing the projector's button located on the upper right corner of the device and listen for beeps to determine if the projector is on/off. If you hear the descending beeps, it means that the projector is in the sleep mode. Try remotely disabling the projector and then re-enabling to see if that resolves the issue.

2. Check the HDMI cable's connection: Mac to the projector and also the adapters' plugged in at either end. If you haven't already, try unpluging and re plugging the HDMI cables to see if it could make the difference.

3. Use different HDMI cable: If you haven't already, try using the station's other HDMI cable. By doing so, it will rule out the issue is not isolated to the particular HDMI cables.

I hope these solutions help you get past the issue quicker. </b>

### Generating 20 samples for evaluation

In [31]:
# Execute the query
response = query_engine.query("Im in room SS1069. My classroom speakers are making a buzzing noise. What should I do?")
display(Markdown(f"<b>{response}</b>"))

<b>Great, thank you for providing more context! Based on the new information, here are three possible solutions for the buzzing noise coming from the classroom speakers in room SS1069:

1. Check the audio output settings on the room PC: If the students are using the room PC, check if the audio output settings are correct. On Windows, you can do this by going to the Control Panel, then to Hardware and Sound, and finally to Audio. From there, ensure that the "Apply echo cancellation" option is not selected.
2. Check for audio output issues with their devices: If the students are using their own devices, check if there are any issues with the audio output. On Mac, go to System Preferences, then to Security & Privacy, and finally to Audio Output. Make sure that the "Use default audio device" option is selected. On Windows, go to Start> Control Panel> Hardware and Sound> Audio and check if the "Balance global audio" option is selected.
3. Check for any physical obstructions in the room: Physical obstructions in the room, such as furniture or equipment, could be causing the buzzing noise.</b>

In [32]:
# Execute the query
response = query_engine.query("Im in room SS1069. My classroom speakers are making a buzzing noise. What should I do?")
display(Markdown(f"<b>{response}</b>"))

<b>Thank you for asking! Based on the information provided, here are max 3 possible solutions for dealing with a buzzing noise from classroom speakers in room SS1069:

1. Check if the podium mic is muted. To do this, use the TalkMaster app to listen in on the room and ensure the instructor is speaking in the room. If the podium is muted, unmute it by clicking the status indicator on the room's status screen and clicking unmute. This can also be done in the Xpansion panel, but it's more invasive, requiring touchscreen control in the room.
2. Verify that the mic level is not too high, as this may cause feedback.  If the mic level is already quite high and an ambient buzz isn't dominating the audio, consider keeping the mic level to avoid potential feedback.
3. Using the Xpansion panel, open the room's setup panel and adjust the speaker settings. Verify that the speaker levels are too high and adjust accordingly to avoid feedback.

It is important to follow instructions in each potential solutions carefully to determine the best course of action for resolving the buzzing</b>

In [33]:
# Execute the query
response = query_engine.query("Im in room BL 116. My classroom speakers are making a buzzing noise. What should I do?")
display(Markdown(f"<b>{response}</b>"))

<b>Sure, I'd be happy to help! Based on the provided answer, here are three possible solutions for the buzzing noise from classroom speakers in room BL 116:

1. Check if the mic level is already quite high and dominated by an ambient buzz. To do this, please listen in on the classroom via the Talksmaster app to ascertain the instructor is speaking within the room.
2. Ensure the podium microphone is not muted. Please check fusion to achieve this.
3. If the room isn't listed within fusion, open the room's XP panel.

Please try these steps and see if they help resolve the buzzing noise issue. If you have any further questions or context, please let me know and I'll be happy to help.</b>

In [34]:
# Execute the query
response = query_engine.query("Im in room UC A101. My classroom speakers are making a buzzing noise. What should I do?")
display(Markdown(f"<b>{response}</b>"))

<b>Ah, you're in Room UC A101 and the classroom speakers are making a buzzing noise.  Here are three possible solutions:

1. Check the power source: Make sure the speakers are properly plugged into a working outlet and that the power cord is securely connected.
2. Reset the speakers: Attempt to reset the speakers by unplugging them and plugging them back in.
3. Check for obstruction: Make certain that nothing is blocking the speakers or the audio output.

If you need further assistance or additional context, please let me know.</b>

In [35]:
# Execute the query
response = query_engine.query("Im in room BA2159. I need batteries for my speakers. What should I do?")
display(Markdown(f"<b>{response}</b>"))

<b>Thank you for providing more context! Based on the information provided, here are three possible solutions for the given query:

1. Check the keyboard tray that pulls out of the front of the teaching station, as well as nearby tables, desks, and ledges. The microphone pack may have fallen off the podium or landed on one of these surfaces.
2. Check underneath cabinets and tables near the podium and in the room. The microphone pack might have rolled beneath something and been overlooked.
3. Check the floor adjacent to the podium and in the room. If the wireless lav mic has fallen off the podium, it might be on the ground.

Please inform me if you need further assistance.</b>

In [36]:
# Execute the query
response = query_engine.query("Im getting a call from room BL 116. What should I do?")
display(Markdown(f"<b>{response}</b>"))

<b>Great, I'm here to help! Based on the new context provided, here are three possible solutions for the issue you're experiencing in Room BL 116:

1. Troubleshoot the issue manually: Since there is a desktop computer attached to the podium connected via VGA, and a switcher under the desk to switch between the desktop and laptop (HDMI), you can try using the mouse and keyboard to navigate the computer and troubleshoot the issue.
2. Use RoomView remote access: Since the room doesn't belong to LSM, but you are assisting with it, you can try using RoomView remote access to troubleshoot the issue.
3. Contact the room's emergency contact Curtis Debi: If all else fails, you can reach out to Curtis Debi outside of his working hours (9am-6pm, Monday to Friday) for assistance

I hope these suggestions are helpful! If you have any further questions or concerns, feel free to ask!</b>

In [37]:
# Execute the query
response = query_engine.query("WO 25 Desktop PC is sent to projector, but no audio coming through. Sounds like he's done everything right, and knows what he's doing.")
display(Markdown(f"<b>{response}</b>"))

<b>Based on the new context provided, here are three possible solutions for the issue of no audio coming through when the WO 25 Desktop PC is sent to the projector:

1. Check the audio output setting on the laptop: Make sure that the audio output from the laptop is set to the correct setting, which should be "Crestron" or "HDMI" depending on the type of laptop.
2. Check the projector settings: Ensure that the projector is set to the correct input and that the audio output is set to the correct setting, which should be "PC Audio".
3. Check the Zoom settings: Verify that the Zoom settings on the laptop are set to the correct output, which should be "Speakers and/or Headphones". If the issue persists, try changing the Zoom output to "System Sound" or "Alternate Audio Device".

I hope these solutions help resolve the issue for the WO 25 Desktop PC! Let me know if additional assistance needed.</b>

In [38]:
# Execute the query
response = query_engine.query("WE 75 unable to send audio through HDMI, laptop speakers only option")
display(Markdown(f"<b>{response}</b>"))

<b>Of course! Based on the new context provided, here's a possible refinement of the original answer:

Hi there! Based on the updated information, here are three possible solutions for the issue of WE 75 unable to send audio through HDMI and only having the laptop speakers as an option:

1. Try unplugging and re-plugging the HDMI cable or adapter from the computer, and then restart the laptop.
2. If the issue persists, check if changing the speaker output to Crestron or HDMI can resolve the issue.
3. If the user is using a Yamaha videoconferencing soundbar, ask them to change their laptop output to Yamaha CS-700.

I hope these suggestions are helpful in resolving the issue!</b>

In [39]:
# Execute the query
response = query_engine.query("Send to all displays doesn't work when Room PC is selected. ")
display(Markdown(f"<b>{response}</b>"))

<b>Okay, I see! Based on the new context, here are three possible solutions for the issue of "Send to all displays doesn't work when Room PC is selected":

1. Check the Room PC settings: Ensure that the Room PC is set to the correct input source and that the audio is not muted.
2. Restart the Room PC: Try restarting the Room PC and then check if the issue is resolved.
3. Check for software updates: Ensure that the Room PC and other devices are operating with the most recent software updates.

I hope this helps! Let me know if you'd like me to refine the answer further.</b>

In [40]:
# Execute the query
response = query_engine.query("SF 2202 Blue screen on crestron")
display(Markdown(f"<b>{response}</b>"))

<b>Based on the new context provided, the most likely cause of the blue screen on the Crestron touchscreen is that the touchscreen is turned off. To turn it back on, you can follow the steps mentioned in the context:

* On the right-hand side of the touchscreen, wrap your hand around the back and feel a vertical column of buttons. The On/Off button is the bottom butt on.
* Press and hold the On/Off button for a short time, and a logo saying "Crestron" should appear after a moment. It will take a moment to boot up, and after that you should be good to go.

Therefore, the possible solutions for the issue of a blue screen on a Crestron touchscreen are:

1. Turn the touchscreen back on by locating the On/Off button and pressing and holding it for a short time.
2. Check and update the touchscreen drivers to ensure they are up to date.
3. Power cycle the touchscreen by unpluging unplugging it for 30 seconds and then plugging it back in.

Please try one of these solutions to see if it resolves the problem. If you</b>

In [41]:
# Execute the query
response = query_engine.query("MS 2158 Can't log in with the touchscreen keyboard—can't figure out how to do underscore. Also needed a walkthrough on how to use the teaching station.")
display(Markdown(f"<b>{response}</b>"))

<b>I apologize for the inconvenience you're experiencing while trying to log in to the teaching station. Here are three possible solutions for the issue you're facing:

1. Remote login assistance: As you're unable to log in to the touchscreen keyboard, I suggest offering to log the user in remotely. This way, they can access the teaching station without any issues.
2. Changing the password associated with the user's account: As mentioned in the medium-term solution, certain keyboard characters (like $ and *) used to be allowed in passwords but are no longer allowed for cybersecurity reasons. If the user is experiencing this issue, I recommend changing the password associated with their account. Once the password is changed, the user will be able to log in to the touchscreens themselves.
3. Providing a walkthrough on how to use the teaching station: For users who are unfamiliar with the teaching station, offering a walkthrough on how to use it can be helpful. This can include step-bystep instructions on how to log in, navigate navigate the platform, and access various features features. By providing clear and concise instructions, the user will be able to use the</b>

In [42]:
# Execute the query
response = query_engine.query("WW 119 Can't get audio over Zoom")
display(Markdown(f"<b>{response}</b>"))

<b>Sure, I'd be happy to help you solve the issue with not being able to get audio over Zoom. Based on the new context provided, here are three possible solutions for the query:

1. Ensure the røde mic is on and unmuted: If using a MacBook or Windows laptop, ensure that the røde microphone is turned on and unmuted. Check the Zoom settings to see if the røde microphone is selected as the audio input source.
2. Unmute the touchscreen wireless microphone source: If using a Windows or Mac laptop, ensure that the touchscreen wireless microphone source is unmuted. Check the Zoom settings to see if the touchscreen microphone source is selected as the audio input source.
3. Restart the Zoom application: If the above steps do not resolve the issue, try restarting the Zoom application and then try again.

I hope these solutions help you resolve the issue with not being able to get audio over Zoom. If you have any further questions or concerns, please feel free to ask.</b>

In [43]:
# Execute the query
response = query_engine.query("FE 230 Can't hear the Zoom speaker and the Zoom speaker can't hear them.")
display(Markdown(f"<b>{response}</b>"))

<b>Sure, I'd be happy to help! Based on the additional context provided, here are three possible solutions for the issue of not being able to hear the Zoom speaker and the opposite problem:

1. Check the microphone settings: Ensure that the microphone the user desires to use is on and unmuted. Additionally, check if the user is experiencing any wireless lav issues by following the steps below.
2. Change the speaker output to a different option: Try changing the laptop output to "crestron", "hdmi", or "np-p401w" to see if that resolves the issue.
3. Verify the laptop image being sent to the projector is the correct one: The issue may be that the last laptop to be sent to the projector is not receiving audio. So, follow the steps above to ensure that the laptop image being sent to the projector is the correct one.

I hope these suggestions help you troubleshoot and resolve the issue. Let me know if you need any more assistance!</b>

In [44]:
# Execute the query
response = query_engine.query("BA B24 video on mac is skipping and fast forwards")
display(Markdown(f"<b>{response}</b>"))

<b>Of course, I'd be happy to help! Based on the new context provided, it seems that the issue of the BA B24 video on Mac skipping and fast forwarding is occurring on newer Macs (released from 2020 onward) . In this case, one possible solution is to try unplugging the HDMI cable from your laptop, and then playing the video or audio again. If their video or audio plays back normally once the HDMI cable is unplugged, you can tell them to restart their laptop with the HDMI cable plugged in. This should make their video or audio play back properly. If the instructors are unenthusiastic about this option, you can inform them that on MacBook models from about 2020 onward, restarting your laptop almost always fixes this particular problem.

Here are three possible solutions for the given issue issue:

1. Check video resolution: Adjust the video resolution to a lower setting and see if that resolves the issue.
2. Update video software: Make sure the video software is up to date, as newer updates may address this issue. Try updating the video software to the latest version.
</b>

In [45]:
# Execute the query
response = query_engine.query("OI5260 Display is off suddenly")
display(Markdown(f"<b>{response}</b>"))

<b>Thank you for providing more context! Based on the existing answer and the new information provided, here are three possible solutions for the issue of the Display is off suddenly:

1. If the projector is in sleep mode, you can try disabling it remotely and then re-enabling it. Alternatively, you can try unplugging and re-plugging the HDMI cable into the laptop, including unplugging any adapters on both ends.
2. If the second (annotation) display is not showing PowerPoint notes, it may have been switched to duplicate desktop. Try switching back to extended desktop to see if that resolves the issue.
3. If the above solutions do not work, try remotely pressing the display on/power button and listening for on or off sounds. If the projector produces descending beeps when turned or ascending beeps when turned off, it may mean the projector is in sleep mode, which can be disabled and then re-enabled.

I hope these revised solutions help! If you have any additional information or context, please let me know and I will be happy to help!</b>

In [46]:
# Execute the query
response = query_engine.query("OI 8200 OCCS feed is not showing anything.")
display(Markdown(f"<b>{response}</b>"))

<b>Based on the provided documents and the context you provide, here are three possible solutions for the issue of the OI 8200 OCCS feed not showing anything:

1. Check if one of the two RTSP links shows content by opening the VLC stream. If none of the links shows content, message the co-pilots for that area to send the content to stream 1, and one of the cameras to stream 2.
2. Check if the RTSP link ending with "extron 1" shows content. If neither of the RTSP links shows content message the co-pilots for the area to send the content to stream 1 and one of the camera to stream 2 using the IP address in the attic.
3. Check if the preview is showing a different source than the one being sent to the projector. The steam takes the source that is being sent to the preview. If the image is giving "input 1 no source," try using the X-Panel to switch to a different preview source, then back immediately. This sometimes "refers" the issue.

I hope this refined answer helps! Let me know if you have any questions</b>

In [47]:
# Execute the query
response = query_engine.query("MY 490 Needed help projecting and who do they need to contact for OCCS")
display(Markdown(f"<b>{response}</b>"))

<b>Thank you for providing more context! Based on the additional information provided, here are three possible solutions for the given query:

1. Co-pilot Check: Ask the co-pilot(s) in the area to check if the room is actually projecting something on the projector. Provide them with the RTSP link so they can easily check via their phone for easy access.
2. Contact OCCS: Inform My 490 that they need to contact OCC (Operations, Capital, and Control Systems) for assistance with projecting. Provide them with the necessary contact information.
3. Check Network: Suggest checking the network connection to ensure it is stable and strong enough for proper projection. If necessary, suggest contacting the IT department for further assistance.

I hope this refined answer helps! Let me know if you have any other questions.</b>

In [48]:
# Execute the query
response = query_engine.query("PB B250 image small on the screen (using HDMI 2)- did not know how to follow resolution settings")
display(Markdown(f"<b>{response}</b>"))

<b>Based on the provided documents and the new context, here are 3 possible solutions to the problem of a small PB B250 image on the screen using HDMI 2:

1. Adjust the Resolution Settings: In the Display Settings menu, select "Crestron" or "HDMI" on the left-hand side list, and then choose one of the resolution options with larger text that produces an image on the big screen. If you get a list of resolutions, choose one of the smaller resolutions that produces an image on the big screen.

2. Use the Menu Bar: At the top-left of the screen, click the Apple icon then select Systems Preferences and choose Display then click the Displays menu and select the desired display then adjust the resolutions settings to increase the image size.

3. Check Display Settings: In the Display menu, click Display Setting option selected the desired from the list and adjust the resolutions settings to increase the image size.

It is worth noting that the best solution may vary depending on the specific situation and the equipment being used. </b>

In [49]:
# Execute the query
response = query_engine.query("GB303 The PC isn't working. I've pressed the physical button, but it's not coming on")
display(Markdown(f"<b>{response}</b>"))

<b>Ah, I see! Here are three possible solutions to help troubleshoot the issue with the PC not working:

1. Check the power button: Make sure that the power button on the PC is functioning properly. Try pressing the button a few times to see if it turns on. If it doesn't, check if the button is faulty or if there's an issue with the power supply.
2. Restart the PC: If the PC is not turning on, try restarting it. unplugging it from the power source for a few seconds and then plugging it back in and see if it turns on. This might fix any temporary power issues.
3. Check for loose connections: Check if any of the cables connecting the PC to the monitor, keyboard and mouse are loose or detached. Restore a stable connection whenever required to confirm if this solves the problem.

I hope these suggestions help you fixed the issue with the PC!</b>

In [50]:
# Execute the query
response = query_engine.query("Con Hall. The device is not conencted to the poidum. Usually it gets connected. can yiu help me out ?")
display(Markdown(f"<b>{response}</b>"))

<b>Thank you for providing more context! Given the new information, here are three possible solutions for the issue of "No audio from source device on room speakers" in the classroom:

1. Check the HDMI cable angle: Ensure that the HDMI cable is connected to the iPad is at a proper angle. The weight of the cable may be pulling it downward, causing the issue. Try adjusting the angle of the cable to see if it helps.
2. Reset the iPad: If the issue persists, try resetting the iPad. This can often help resolve compatibility issues. To reset the iPad, go to Settings > General > Reset > Erase all content and settings. Note that this will erase all content and settings, so be sure to enter the correct information after the reset is complete.
3. Check the Crestron software settings: If using a Crestron device, ensure that the Crestron software settings are properly configured. Check that the correct output device is selected and the volume is turned up high enough. Try adjusting these settings to see if it helps resolve the issue.

I hope this helps! Let me know if you have any other questions.</b>